In [1]:
import os
import pandas as pd
import datetime as dt
import time
import calendar
import numpy as np
from statistics import mean

from uce_resources import get_site_id, get_mms_data, get_applied_forecast, get_prices, get_green_tariff
from uce_resources import make_results

In [2]:
from settings.sites import ceg as sites_list

target_year = 2022
target_month = 1
forecasts_types = ['real', 'naive', 'zero']

target_folder = 'data/enercast_real_data/'.format(target_year, target_month)
if not os.path.exists(target_folder):
    os.makedirs(target_folder)

sites_list = ['Vasylivka']
sites_data = dict.fromkeys(sites_list)
print(sites_data)

{'Vasylivka': None}


In [3]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.pool import NullPool
from settings.db import DO_URL

engine = create_engine(DO_URL, poolclass=NullPool)
metadata = MetaData()
metadata.reflect(bind=engine)

C:\Users\dmytro\AppData\Local\Temp\ipykernel_23508\3375350544.py:7: SAWarning: Did not recognize type 'point' of column 'location'
  metadata.reflect(bind=engine)


In [4]:
with engine.connect() as connection:
        
    for site in sites_data.keys():
        start = time.time()
        print('-'*50)
        print(site)
        site_data = dict()
        site_data['site'] = site
        site_data['site_id'], site_data['legal_entity'] = get_site_id(site, connection, 
                                                                      metadata.tables['sites'],
                                                                      include_legal_entity_id=True)
        
        mms_data, site_data['mms_version'] = get_mms_data(site_data['site_id'], 
                                                          target_year, target_month, 
                                                          connection, metadata.tables['mms_data'], include_prev=False)
        
        print(mms_data)

--------------------------------------------------
Vasylivka
                     yield [kWh]
2021-12-31 22:30:00         -140
2021-12-31 23:30:00         -135
2022-01-01 00:30:00         -135
2022-01-01 01:30:00         -133
2022-01-01 02:30:00         -139
...                          ...
2022-01-31 17:30:00         -133
2022-01-31 18:30:00         -131
2022-01-31 19:30:00         -137
2022-01-31 20:30:00         -138
2022-01-31 21:30:00         -143

[744 rows x 1 columns]
